### Query Expansion
In this section we will cover a retrieval optimization technique called query expansion. This involves adding more context and tags to the user query to be able to produce better more accurate outputs